<a href="https://colab.research.google.com/github/sanjeev1996/OpenCV/blob/master/Age_and_Gender_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pafy
!pip install youtube_dl

In [0]:
import cv2
import numpy as np
import pafy

url = 'https://www.youtube.com/watch?v=c07IsbSNqfI&feature=youtu.be'
vpafy = pafy.new(url)
play = vpafy.getbest(preftype='mp4')

cap = cv2.VideoCapture(play.url)

cap.set(3,480)
cap.set(4, 460)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']


def load_caffe_models():
  age_net = cv2.dnn.readNetFromCaffe('deploy_age.prototxt', 'age_net.caffemodel')
  gender_net = cv2.dnn.readNetFromCaffe('deploy_gender.prototxt', 'gender_net.caffemodel')
  return (age_net, gender_net)

def video_detector(age_net, gender_net):
  font = cv2.FONT_HERSHEY_SIMPLEX

  while True:
    ret, image = cap.read()

    face_cascade = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if (len(faces)>0):
      print("Found {} faces".format(str(len(faces))))

    for (x,y,w,h) in faces:
      cv2.rectangle(image, (x,y), (x+w, y+h), (255,255,0), 2)
    
    # Crop face from image
    face_img = image(y:y+h, h:h+w).copy()
    blob = cv2.dnnblobFromImage(face_img, 1, (277, 277), MODEL_MEAN_VALUES, swapRB=False)

    # Predict Gender
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_list[gender_preds[0].argmax()]
    print("Gender: "+ gender)

    # Predict age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = age_list[age_preds[0].argmax()]
    print("Age Range: "+age)

    overlay_text = "%s %s" % (gender,age)
    cv2.putText(image, overlay_text, (x,y), font, 1, (255,255,255), 2, cv2.LINE_AA)
    cv2_imshow(image)
    if cv2.waitkey(1) & 0xFF == ord('q'):
      break
     

if __name__ == "__main__":
  age_net, gender_net = load_caffe_models()
  video_detector(age_net, gender_net)


